# ビジネス関連法案の RAG 検索 

In [ ]:
import logging
import cohere
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from typing import List, Dict
import time
import re
from dotenv import load_dotenv
import os
import uuid
import hnswlib

In [ ]:
load_dotenv()
co = cohere.ClientV2(os.environ['COHERE_APIKEY'])

EGOV_SEARCH_URL = "https://elaws.e-gov.go.jp/api/1/lawlists/1"

# 2. e-Gov から法令情報を取得

In [ ]:
documents = [
    {
        'title': '昭和六十三年法律第百八号 消費税法',
        'url': 'https://laws.e-gov.go.jp/api/1/lawdata/363AC0000000108', #https://laws.e-gov.go.jp/law/363AC0000000108'
    },
    #{
    #    'title': '昭和四十年法律第三十三号 所得税法',
    #    'url': 'https://laws.e-gov.go.jp/api/1/lawdata/340AC0000000033'
    #},
    #{
    #    'title': '昭和二十八年法律第六号 酒税法',
    #    'url': 'https://laws.e-gov.go.jp/api/1/lawdata/328AC0000000006'
    #},        
]
# 429: trial token rate limit exceeded, limit is 100000 tokens per minute

documents = [
    {
        'title': '国税庁 消費税のしくみ',
        'url': 'https://www.nta.go.jp/publication/pamph/koho/kurashi/html/01_3.htm'
    },
]


In [ ]:
# From LLMU

from unstructured.partition.html import partition_html
from unstructured.partition.xml import partition_xml
from unstructured.chunking.title import chunk_by_title

class Vectorstore:
    """
    A class representing a collection of documents indexed into a vectorstore.

    Parameters:
    raw_documents (list): A list of dictionaries representing the sources of the raw documents. Each dictionary should have 'title' and 'url' keys.

    Attributes:
    raw_documents (list): A list of dictionaries representing the raw documents.
    docs (list): A list of dictionaries representing the chunked documents, with 'title', 'text', and 'url' keys.
    docs_embs (list): A list of the associated embeddings for the document chunks.
    docs_len (int): The number of document chunks in the collection.
    idx (hnswlib.Index): The index used for document retrieval.

    Methods:
    load_and_chunk(): Loads the data from the sources and partitions the HTML content into chunks.
    embed(): Embeds the document chunks using the Cohere API.
    index(): Indexes the document chunks for efficient retrieval.
    retrieve(): Retrieves document chunks based on the given query.
    """

    def __init__(self, raw_documents: List[Dict[str, str]]):
        self.raw_documents = raw_documents
        self.docs = []
        self.docs_embs = []
        self.retrieve_top_k = 10
        self.rerank_top_k = 3
        self.load_and_chunk()
        self.embed()
        self.index()


    def load_and_chunk(self) -> None:
        """
        Loads the text from the sources and chunks the HTML content.
        """
        print("Loading documents...")

        for raw_document in self.raw_documents:
            if raw_document['url'].endswith('.htm') or raw_document['url'].endswith('.html'):
                elements = partition_html(url=raw_document['url'])
            else:        
                filename = f"{raw_document['title']}.xml"
                with open(filename, "w") as f:
                    res = requests.get(raw_document["url"])
                    f.write(res.text)
            
                elements = partition_xml(filename=filename)
            chunks = chunk_by_title(elements)
            for chunk in chunks:
                self.docs.append(
                    {
                        "title": raw_document["title"],
                        "text": str(chunk),
                        "url": raw_document["url"],
                    }
                )

    def embed(self) -> None:
        """
        Embeds the document chunks using the Cohere API.
        """
        print("Embedding document chunks...")

        batch_size = 90
        self.docs_len = len(self.docs)
        for i in range(0, self.docs_len, batch_size):
            batch = self.docs[i : min(i + batch_size, self.docs_len)]
            texts = [item["text"] for item in batch]
            docs_embs_batch = co.embed(
                texts=texts, model="embed-v4.0", input_type="search_document"
            ).embeddings
            self.docs_embs.extend(docs_embs_batch)

    def index(self) -> None:
        """
        Indexes the document chunks for efficient retrieval.
        """
        print("Indexing document chunks...")

        self.idx = hnswlib.Index(space="ip", dim=1024)
        self.idx.init_index(max_elements=self.docs_len, ef_construction=512, M=64)
        print(list(range(len(self.docs_embs))))
        self.idx.add_items(self.docs_embs, list(range(len(self.docs_embs))))

        print(f"Indexing complete with {self.idx.get_current_count()} document chunks.")

    def retrieve(self, query: str) -> List[Dict[str, str]]:
        """
        Retrieves document chunks based on the given query.

        Parameters:
        query (str): The query to retrieve document chunks for.

        Returns:
        List[Dict[str, str]]: A list of dictionaries representing the retrieved document chunks, with 'title', 'text', and 'url' keys.
        """

        # Dense retrieval
        query_emb = co.embed(
            texts=[query], model="embed-v4.0", input_type="search_query"
        ).embeddings
        
        doc_ids = self.idx.knn_query(query_emb, k=self.retrieve_top_k)[0][0]

        # Reranking
        rank_fields = ["title", "text"] # We'll use the title and text fields for reranking

        docs_to_rerank = [self.docs[doc_id] for doc_id in doc_ids]
        rerank_results = co.rerank(
            query=query,
            documents=docs_to_rerank,
            top_n=self.rerank_top_k,
            model="rerank-english-v3.0",
            rank_fields=rank_fields
        )

        doc_ids_reranked = [doc_ids[result.index] for result in rerank_results.results]

        docs_retrieved = []
        for doc_id in doc_ids_reranked:
            docs_retrieved.append(
                {
                    "title": self.docs[doc_id]["title"],
                    "text": self.docs[doc_id]["text"],
                    "url": self.docs[doc_id]["url"],
                }
            )

        return docs_retrieved

In [ ]:
vectorstore = Vectorstore(documents)

In [ ]:
message = '公務員の義務はどれに該当する？'

response = co.chat_stream(
    messages=[
        {
            "role": "user",
            "content": message,
        }],
    model="command-a-03-2025",
    documents=documents)

In [ ]:
for event in response:
    if event.type == "content-delta":
        print(event.delta.message.content.text, end="")

In [ ]:
def create_embeddings(texts: List[str], batch_size: int = 96) -> np.ndarray:
    """
    テキストリストをCohere APIでベクトル化
    
    Args:
        texts: ベクトル化するテキストのリスト
        batch_size: バッチサイズ
    
    Returns:
        埋め込みベクトルの配列
    """
    embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        try:
            response = co.embed(
                texts=batch,
                model='embed-multilingual-v3.0',
                input_type='search_document'
            )
            #logging.warning(response.embeddings)
            embeddings.extend(response.embeddings)
            print(f"✓ {i+len(batch)}/{len(texts)} 件処理完了")
            time.sleep(0.5)  # レート制限対策
        except Exception as e:
            print(f"⚠ エラー (batch {i}): {e}")
            break
    return np.array(embeddings, dtype=object)

# 法令名をベクトル化
print("\n法令情報をベクトル化中...")
law_texts = df_laws['text'].tolist()
law_embeddings = create_embeddings(law_texts)

print(f"✓ ベクトル化完了: {law_embeddings.shape}")

In [ ]:
# =============================================================================
# 4. RAG検索関数の実装
# =============================================================================

def semantic_search(query: str, top_k: int = 10) -> pd.DataFrame:
    """
    セマンティック検索を実行
    
    Args:
        query: 検索クエリ
        top_k: 上位何件を返すか
    
    Returns:
        検索結果のデータフレーム
    """
    # クエリをベクトル化
    query_response = co.embed(
        texts=[query],
        model='embed-multilingual-v3.0',
        input_type='search_query'
    )
    query_embedding = np.array(query_response.embeddings["float"][0])
    
    # コサイン類似度を計算
    similarities = np.dot(law_embeddings, query_embedding) / (
        np.linalg.norm(law_embeddings, axis=1) * np.linalg.norm(query_embedding)
    )
    
    # 上位k件を取得
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    results = df_laws.iloc[top_indices].copy()
    results['similarity_score'] = similarities[top_indices]
    
    return results

def rerank_results(query: str, results: pd.DataFrame, top_k: int = 5) -> pd.DataFrame:
    """
    Cohere Rerank APIで結果を再ランキング
    
    Args:
        query: 検索クエリ
        results: 初期検索結果
        top_k: 最終的に返す件数
    
    Returns:
        再ランキング後の結果
    """
    documents = results['text'].tolist()
    
    try:
        rerank_response = co.rerank(
            query=query,
            documents=documents,
            model='rerank-multilingual-v3.0',
            top_n=top_k
        )
        
        # 再ランキング結果を整理
        reranked_indices = [r.index for r in rerank_response.results]
        reranked_scores = [r.relevance_score for r in rerank_response.results]
        
        reranked_results = results.iloc[reranked_indices].copy()
        reranked_results['rerank_score'] = reranked_scores
        
        return reranked_results
        
    except Exception as e:
        print(f"⚠ Rerank エラー: {e}")
        return results.head(top_k)


# サンプル実行

In [ ]:
business_queries = [
    "会社設立や株式会社の運営に関する法律",
    "従業員の雇用や労働条件に関する法律",
    "個人情報やデータ保護に関する法律",
    "契約や取引に関する法律",
    "知的財産権に関する法律"
]

all_business_laws = []

for query in business_queries:
    print(f"\n🔍 検索: {query}")
    print("-" * 70)
    
    # セマンティック検索
    search_results = semantic_search(query, top_k=10)
    """
    # Rerankで精度向上
    final_results = rerank_results(query, search_results, top_k=3)
    
    # 結果を表示
    for idx, row in final_results.iterrows():
        print(f"  • {row['law_name']}")
        print(f"    法令番号: {row['law_number']}")
        print(f"    関連度: {row.get('rerank_score', row.get('similarity_score', 0)):.3f}")
        print()
        
        all_business_laws.append({
            'query': query,
            'law_name': row['law_name'],
            'law_number': row['law_number'],
            'score': row.get('rerank_score', row.get('similarity_score', 0))
        })
    """


(まだ途中)